In [21]:
import requests
import json
import time
import os

API_KEY = 'ZdpeG8xFeTZRSqqiNnbe6tvcV9OI6FKenEdeuEpMLuUhq41DQ9EX9-71yKGfsIvxW9mYIzqZznCG2AuECKuQVOmGUvti485q_e_dS4OxSi3ZCSqwcABXYhdjyr9oZnYx'
HEADERS = {'Authorization': f'Bearer {API_KEY}'}
URL = 'https://api.yelp.com/v3/businesses/search'
DATABASE_FILE = 'restaurants_data.json'

def fetch_yelp_data(location, offset):
    params = {
        'term': 'restaurants',
        'location': location,
        'limit': 50,
        'offset': offset
    }
    response = requests.get(URL, headers=HEADERS, params=params)
    return response.json()

def write_to_file(data):

    if not os.path.exists(DATABASE_FILE):
        with open(DATABASE_FILE, 'r') as file:
            file_data = json.load(file)
            existing_business_ids = {business['id'] for business in file_data['businesses']}
    else:
        file_data = {'businesses': []}
        existing_business_ids = set()

    new_businesses = [business for business in data['businesses'] if business['id'] not in existing_business_ids]

    if new_businesses:
        # Append new businesses to the existing data
        file_data['businesses'].extend(new_businesses)

        # Write updated data back to the file
        with open(DATABASE_FILE, 'w') as file:
            json.dump(file_data, file, indent=4)

locations = [
    "San Francisco, CA", "Oakland, CA", "Berkeley, CA", "San Jose, CA",
    "Sacramento, CA", "Stockton, CA", "Fremont, CA", "Cupertino, CA",
    "Pacifica, CA", "Fresno, CA", "Modesto, CA", "Alameda, CA"
]

for location in locations:
    offset = 0

    while True:
        data = fetch_yelp_data(location, offset)
        if not data.get('businesses'):
            print("Not valid data")
            print(data)
            offset += 50
            break

        write_to_file(data)
        offset += 50
        time.sleep(1)

        if offset >= data['total']:
            print("No more data")
            break

    print("Moving to next location")

print("Finished fetching data")

Not valid data
{'error': {'code': 'INTERNAL_ERROR', 'description': 'Something went wrong internally, please try again later.'}}
Not valid data
{'error': {'code': 'INTERNAL_ERROR', 'description': 'Something went wrong internally, please try again later.'}}
Not valid data
{'error': {'code': 'VALIDATION_ERROR', 'description': 'Too many results requested, limit+offset must be <= 1000.'}}
Not valid data
{'error': {'code': 'INTERNAL_ERROR', 'description': 'Something went wrong internally, please try again later.'}}
Not valid data
{'error': {'code': 'VALIDATION_ERROR', 'description': 'Too many results requested, limit+offset must be <= 1000.'}}
No more data
Not valid data
{'error': {'code': 'ACCESS_LIMIT_REACHED', 'description': "You've reached the access limit for this client. See instructions for requesting a higher access limit at https://docs.developer.yelp.com/docs/fusion-rate-limiting"}}
Not valid data
{'error': {'code': 'ACCESS_LIMIT_REACHED', 'description': "You've reached the access 

In [23]:
import pandas as pd

def parse_yelp_data(json_data):
    restaurants = []
    for business in json_data['businesses']:
        if 'price' in business:

            restaurant = {
                'name': business['name'],
                'cuisine': ', '.join([category['title'] for category in business['categories']]),
                'city' : business['location']['city'],
                'address': business['location']['address1'],
                'price_range': business.get('price', 'N/A'),
                'average_rating': business['rating'],
                'review_count': business['review_count']
            }
            restaurants.append(restaurant)
    return restaurants


parsed_data = []
with open(DATABASE_FILE, 'r') as file:
    for line in file:
        json_data = json.loads(line)
        parsed_data.extend(parse_yelp_data(json_data))

restaurants_df = pd.DataFrame(parsed_data)


print(restaurants_df.shape)

restaurants_df.head()

(4843, 7)


,name,cuisine,city,address,price_range,average_rating,review_count
0,Bottega,"Italian, Pasta Shops, Pizza",San Francisco,1132 Valencia St,$$,4.3,1119
1,Fog Harbor Fish House,"Seafood, Wine Bars, Cocktail Bars",San Francisco,39 Pier,$$,4.4,10498
2,Horsefeather,"Cocktail Bars, New American",San Francisco,528 Divisadero St,$$,4.0,605
3,Santeria,"Mexican, Cocktail Bars",San Francisco,2251 Market St,$$,4.4,334
4,Marufuku Ramen,Ramen,San Francisco,1581 Webster St,$$,4.5,5153


In [25]:
duplicate_rows = restaurants_df[restaurants_df.duplicated()]
duplicate_rows.head()

,name,cuisine,city,address,price_range,average_rating,review_count
756,Fog Harbor Fish House,"Seafood, Wine Bars, Cocktail Bars",San Francisco,39 Pier,$$,4.4,10498
770,Bottega,"Italian, Pasta Shops, Pizza",San Francisco,1132 Valencia St,$$,4.3,1119
777,Cafe Okawari,"Japanese, Cafes",San Francisco,236 Townsend St,$$,4.5,408
824,a Mano,"Italian, Cocktail Bars, Pizza",San Francisco,450 Hayes St,$$$,4.0,1693
835,Marufuku Ramen,Ramen,San Francisco,1581 Webster St,$$,4.5,5153
